# Outlier Analysis - KPK Survey 2025

Notebook ini berisi analisis outlier pada data survey yang telah berjalan

In [11]:
import pandas as pd
import numpy as np

df = pd.read_excel('copy-3-response-kpk-survey.xlsx')
print(f"Total respondents: {len(df)}")
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

Total respondents: 910


## 1: Berdasarkan Konsumsi Media

**Rationale:** Memahami bagaimana penggunaan media mempengaruhi persepsi tertentu

**Kriteria:**
a. Memiliki tingkat konsumsi media yang sangat tinggi, berdasarkan durasi
konsumsi
b. Memiliki sumber infromasi yang sangat beragam dalam mendapatkan
infromasi mengenai isu politik
c. Sangat sering menemui atau terpapar infromasi mengenai KPK dalam
mengkonsumsi media

In [12]:
""""
1a. Memiliki tingkat konsumsi media yang sangat tinggi, 
berdasarkan durasi konsumsi

"""
# --- 1a. Memiliki tingkat konsumsi media yang sangat tinggi ---
# Logika:
# - Mengambil durasi penggunaan 6 jenis media (cetak, TV, radio, medsos, podcast, email newsletter).
# - Setiap kategori durasi diubah menjadi nilai numerik (0.5 sampai 10 jam).
# - Total durasi dihitung dengan menjumlahkan seluruh durasi media.
#
# Output utama disimpan di kolom:
#   -> 'total_durasi'
#
# Outlier:
#   -> Responden dengan total durasi tertinggi (top 10 setelah sorting).
#
# Filter tambahan:
#   -> Hanya responden yang menjawab "Ya, bersedia" pada kolom kesediaan dihubungi.

'"\n1a. Memiliki tingkat konsumsi media yang sangat tinggi, \nberdasarkan durasi konsumsi\n\n'

In [13]:
duration_cols = [
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Media Cetak (Koran, Majalah, dan sejenisnya)]',
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Televisi]',
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Radio]',
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Media Sosial]',
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Siniar/Podcast]',
    '3. Berapa rata-rata total durasi (perkiraan) Anda menggunakan media berikut dalam satu hari? [Email Newsletter]'
]

duration_map = {
    '< 1 jam': 0.5,
    '1 - 3 jam': 2,
    '4 - 6 jam': 5,
    '7 - 9 jam': 8,
    '> 9 jam': 10
}

df_duration = df[duration_cols].replace(duration_map)
df_duration = df_duration.apply(pd.to_numeric, errors='coerce').fillna(0)

# Outliers
df['total_durasi'] = df_duration.sum(axis=1)
df_sorted = df.sort_values(by='total_durasi', ascending=False)

# Filter Willingness
contact_col = 'Apakah Anda bersedia dihubungi kembali untuk diskusi lanjutan terkait jawaban survei ini (jika diperlukan)?'

# Outliers dan Willingness
outliers_1a = df_sorted.copy()
outliers_1a_contactable = outliers_1a[outliers_1a[contact_col] == 'Ya, bersedia']

print("Konsumsi Media Tertinggi dan Bersedia Dihubungi:")
outliers_1a_contactable.head(20)
outliers_1a_contactable.to_excel('outliers_1a.xlsx', index=False)

Konsumsi Media Tertinggi dan Bersedia Dihubungi:


In [14]:
"""
1b. Memiliki sumber infromasi yang sangat beragam dalam mendapatkan
infromasi mengenai isu politik
"""

# --- 1b. Memiliki sumber informasi yang sangat beragam dalam isu politik ---
# Logika:
# - Mengambil frekuensi konsumsi 6 jenis media (cetak, TV, radio, medsos, podcast, email newsletter).
# - Setiap kategori frekuensi dikonversi menjadi angka (0 sampai 5).
# - Seseorang dianggap "aktif" pada sebuah media jika frekuensi >= 3 (minimal 'Sekali seminggu').
# - Jumlah media aktif disimpan pada kolom:
#       -> 'diverse_sources_count'
#
# Outlier:
# - Responden yang MENGONSUMSI semua 6 media secara rutin (count == 6).
#
# Filter tambahan:
# - Hanya responden yang menjawab "Ya, bersedia" pada kolom kesediaan dihubungi.


'\n1b. Memiliki sumber infromasi yang sangat beragam dalam mendapatkan\ninfromasi mengenai isu politik\n'

In [15]:
freq_cols = [
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [Media Cetak (Koran, Majalah, dan sejenisnya)]',
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [TV]',
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [Radio]',
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [Media Sosial]',
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [Siniar/Podcast]',
    '1. Secara umum, seberapa sering Anda mengonsumsi media-media berikut? [Email Newsletter]'
]
freq_map = {
    'Tidak pernah sama sekali': 0,
    'Sangat jarang': 1,
    'Beberapa kali sebulan': 2,
    
    'Sekali seminggu': 3,
    'Beberapa kali seminggu': 4,
    'Setiap hari': 5
}

# Covert
df_freq = df[freq_cols].replace(freq_map)
df_freq = df_freq.apply(pd.to_numeric, errors='coerce').fillna(0)

# Active Flag (>= Sekali seminggu) - Baseline untuk bilang seseorang sering mengonsumsi media
df['diverse_sources_count'] = (df_freq >= 3).sum(axis=1)
df = df.sort_values(by='diverse_sources_count', ascending=False)

# Outliers dan Willingness
outliers_1b = df[df['diverse_sources_count'] == len(freq_cols)].copy()
outliers_1b_contactable = outliers_1b[outliers_1b[contact_col] == 'Ya, bersedia']

print("Outlier dengan sumber infromasi yang sangat beragam dan Bersedia Dihubungi:")
outliers_1b_contactable.head(20)

outliers_1b_contactable.to_excel('outliers_1b.xlsx', index=False)

Outlier dengan sumber infromasi yang sangat beragam dan Bersedia Dihubungi:


In [16]:
"""
1c. Sangat sering menemui atau terpapar informasi mengenai KPK
"""
# Logika:
# - Mengambil frekuensi paparan informasi KPK dalam 3 bulan terakhir.
# - Jawaban dikonversi menjadi skor numerik pada kolom:
#       -> 'kpk_exposure_score'
# - Outlier didefinisikan sebagai responden yang menjawab:
#       -> 'Setiap hari'
#
# Filter tambahan:
# - Hanya responden yang menjawab "Ya, bersedia" pada kolom kesediaan dihubungi.
# - Hasil akhir disortir berdasarkan skor paparan KPK (kpk_exposure_score) secara descending.

'\n1c. Sangat sering menemui atau terpapar informasi mengenai KPK\n'

In [17]:
kpk_exposure_col = '3. Seberapa sering Anda melihat, membaca, atau mendengar pemberitaan atau informasi mengenai KPK dalam 3 bulan terakhir?'

# numeric mapping
kpk_map = {
    'Setiap hari': 5,
    'Beberapa kali seminggu': 4,
    'Sekali seminggu': 3,
    'Beberapa kali sebulan': 2,
    'Sangat jarang': 1,
    'Tidak pernah sama sekali': 0
}

# add numeric score
df['kpk_exposure_score'] = df[kpk_exposure_col].map(kpk_map).fillna(0)

# Outlier definition
exposure_active_flag = ['Setiap hari']

# Filter outliers + contactable
outliers_1c = df[df[kpk_exposure_col].isin(exposure_active_flag)].copy()
outliers_1c_contactable = outliers_1c[outliers_1c[contact_col] == 'Ya, bersedia']

# Sort
outliers_1c_contactable = outliers_1c_contactable.sort_values(
    by='kpk_exposure_score',
    ascending=False
)

print("Outliers 1c (Frequent KPK Exposure):")
outliers_1c_contactable.head(20)
outliers_1c_contactable.to_excel('outliers_1c.xlsx', index=False)

Outliers 1c (Frequent KPK Exposure):


## 2: Pengetahuan terhadap kanal media komunikasi KPK

**Rationale:** Dibutuhkan pengetahuan dalam level tertentu dalam memiliki
pendapat mengenai kanal media komunikasi milik KPK

**Kriteria:**
a. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK.

In [18]:
"""
2 a1. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK

"""
# --- 2a1. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK ---
# Logika:
# - Mengambil data dari kolom pilihan kanal komunikasi resmi KPK.
# - Setiap responden mendapatkan skor pada kolom:
#       -> 'kpk_official_score'
#   dengan aturan:
#       - 1 jika responden mengetahui/pernah mengunjungi kanal (checked)
#       - 0 jika tidak
#
# Outlier criteria:
# - Responden dianggap outlier bila:
#       -> kpk_official_score >= 3
#
# Filter tambahan:
# - Hanya menampilkan responden yang menjawab "Ya, bersedia" untuk dihubungi kembali.
#
# Output kolom penting:
#   - 'kpk_official_score' : Jumlah kanal KPK yang diketahui/dikunjungi responden

'\n2 a1. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK\n\n'

In [19]:
kpk_official_cols = [
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [KPK (Official) - Biro Humas]"
]

def is_checked(val):
    if pd.isna(val):
        return 0
    if str(val).strip().lower() in ["Tidak ada yang saya ketahui", "0", ""]:
        return 0
    return 1

# kriteria outlier dan kesediaan
df['kpk_official_score'] = df[kpk_official_cols].apply(lambda x: x.map(is_checked)).sum(axis=1)
outliers_2a1 = df[df['kpk_official_score'] >= 3].copy()
outliers_2a1

outliers_2a1_contactable = outliers_2a1[outliers_2a1[contact_col] == "Ya, bersedia"]

print(f"Total Outliers: {len(outliers_2a1)}")
print(f"Outliers Contactable: {len(outliers_2a1_contactable)}")

outliers_2a1_contactable.head(20)
outliers_2a1_contactable.to_excel('outliers_2a1.xlsx', index=False)

Total Outliers: 0
Outliers Contactable: 0


In [20]:
"""
2 a2. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK
"""
# Logika:
# - Menggunakan seluruh daftar kanal KPK (10 kanal resmi).
# - Setiap kanal memiliki beberapa platform (IG, X, FB, TikTok, YouTube, dll),
#   sehingga perlu dicek per baris apakah responden mengetahui minimal satu platform.
#
# Output kolom utama:
#   - 'kpk_kanal_count' :
#         Jumlah kanal KPK yang diketahui responden (0–10),
#         dihitung berdasarkan:
#             -> 1 jika responden mengetahui minimal satu platform dalam kanal tsb
#             -> 0 jika tidak mengetahui apa pun
#
# Outlier criteria:
#   - Responden dianggap outlier bila:
#         -> kpk_kanal_count == 10  (mengetahui SEMUA kanal KPK)
#
# Filter tambahan:
#   - Hanya menampilkan responden yang menjawab:
#         -> "Ya, bersedia" pada kolom contact_col
#
# Output final:
#   - Tabel berisi responden outlier
#   - Disortir descending berdasarkan kolom 'kpk_kanal_count'

'\n2 a2. Mengetahui dan/atau mengikuti banyak kanal media komunikasi milik KPK\n'

In [21]:
kpk_official_cols = [
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [KPK (Official) - Biro Humas]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Jaga - Direktorat Gratifikasi]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Literasi Gratifikasi - Direktorat Gratifikasi]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [ACLC - Direktorat ACLC]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Jejaring Pendidikan - Direktorat Jejaring Pendidikan]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Sosialisasi & Kampanye - Direktorat Sosialisasi dan Kampanye]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [STRANAS PK]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [LSP KPK]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Peran Serta Masyarakat]",
    "9. Dari program, direktorat, atau pengelola berikut, media sosial mana saja yang Anda ketahui dan pernah kunjungi? (Silakan centang semua yang sesuai)   [Lelang Barang Rampasan  - Direktorat Labuksi]"
]

# Build row groups: find all platform columns for each kanal row
row_groups = {}
for kanal in kpk_official_cols:
    row_groups[kanal] = [col for col in df.columns if col.startswith(kanal)]

# Function: 1 if any platform checked, else 0
def kanal_known(row):
    if row.isna().all():
        return 0
    if (row == "Tidak ada yang saya ketahui").any():
        return 0
    return 1

# Count how many kanal a person knows
df['kpk_kanal_count'] = 0
for kanal, cols in row_groups.items():
    df['kpk_kanal_count'] += df[cols].apply(kanal_known, axis=1)

# Outlier threshold MAX ALL
THRESHOLD_2A2 = 10

outliers_2a2 = df[df['kpk_kanal_count'] >= THRESHOLD_2A2].copy()
outliers_2a2_contactable = outliers_2a2[outliers_2a2[contact_col] == "Ya, bersedia"]
df_sorted_kanal = df.sort_values(by='kpk_kanal_count', ascending=False)

outliers_2a2_contactable.head(20)
outliers_2a2_contactable.to_excel('outliers_2a2.xlsx', index=False)


## 3: Pengetahuan dan persepsi mengenai KPK

**Rationale:** Dapat mengethaui sumber dari kepercayaan terhadap KPK
sebagai lembaga pemerintah yang bergerak di bidang pemberantasan korupsi dengan
infroman yang memiliki persepsi tertentu yang terbentuk oleh pengetahuannya
mengenai program KPK

**Kriteria:**
a. Memiliki tingkat kepercayaan yang amat tinggi maupun amat rendah. Percaya
bahwa KPK adalah lembaga yang amat bisa dipercaya atau amat tidak bisa
dipercaya
b. Mengetahui, merasakan, atau mengikuti banyak program
pencegahan/pemberantasan korupsi oleh KPK

In [22]:
"""
3a. Extreme Trust (Tingkat Kepercayaan Sangat Tinggi)
"""

'\n3a. Extreme Trust (Tingkat Kepercayaan Sangat Tinggi)\n'

In [23]:
# 3a-1. Extreme Trust (Highest / Sangat Percaya)
# Logika:
# - Mengambil data survei kepercayaan terhadap KPK.
# - Mengonversi input menjadi numerik (skala 1-6).
# - Menargetkan responden yang memberikan skor maksimum (Very Positive).
#
# Output kolom utama:
#   - 'trust_score' :
#          Nilai numerik tingkat kepercayaan (1-6).
#
# Outlier criteria:
#   - Responden dianggap outlier bila:
#          -> trust_score == 6 (Memberikan nilai kepercayaan tertinggi)
#
# Filter tambahan:
#   - Hanya menampilkan responden yang menjawab:
#          -> "Ya, bersedia" pada kolom contact_col
#
# Output final:
#   - Tabel berisi responden pendukung berat (promoter).
#   - Disortir descending berdasarkan 'trust_score'.

In [24]:
trust_col = '2. Secara keseluruhan, seberapa besar tingkat kepercayaan Anda terhadap lembaga KPK saat ini?'
df['trust_score'] = pd.to_numeric(df[trust_col], errors='coerce')
TARGET_HIGH = 6
trust_high = df[df['trust_score'] == TARGET_HIGH].copy()

# Trust and Willingness
trust_high_contactable = trust_high[trust_high[contact_col] == "Ya, bersedia"]
trust_high_contactable = trust_high_contactable.sort_values(by='trust_score', ascending=False)

trust_high_contactable.head(20)
trust_high_contactable.to_excel('outliers_3a1_highest.xlsx', index=False)

In [25]:
# 3a-2. Extreme Trust (Lowest / Sangat Tidak Percaya)
# Logika:
# - Mengambil data survei kepercayaan terhadap KPK.
# - Mengonversi input menjadi numerik (skala 1-6).
# - Menargetkan responden yang memberikan skor minimum (Very Negative).
#
# Output kolom utama:
#   - 'trust_score' :
#          Nilai numerik tingkat kepercayaan (1-6).
#
# Outlier criteria:
#   - Responden dianggap outlier bila:
#          -> trust_score == 1 (Memberikan nilai kepercayaan terendah)
#
# Filter tambahan:
#   - Hanya menampilkan responden yang menjawab:
#          -> "Ya, bersedia" pada kolom contact_col
#
# Output final:
#   - Tabel berisi responden yang sangat kritis/skeptis (detractor).
#   - Disortir ascending berdasarkan 'trust_score' (prioritas nilai 1)

In [26]:
trust_col = '2. Secara keseluruhan, seberapa besar tingkat kepercayaan Anda terhadap lembaga KPK saat ini?'

df['trust_score'] = pd.to_numeric(df[trust_col], errors='coerce')
TARGET_LOW = 1

trust_low = df[df['trust_score'] == TARGET_LOW].copy()

# Only those willing to be contacted
trust_low_contactable = trust_low[trust_low[contact_col] == "Ya, bersedia"]
trust_low_contactable = trust_low_contactable.sort_values(by='trust_score', ascending=True)

trust_low_contactable.head(20)
trust_low_contactable.to_excel('outliers_3a2_lowest.xlsx', index=False)

## 4: Trust Radar

**Rationale:** Memberikan perspektif unik mengenai faktor kontekstual yang
membentuk persepsi tersebut.

**Kriteria:** \
a. Memiliki kecenderungan spesifik yang extreme (misal: memiliki tingkat
persepsi terhadap kompetensi KPK yang sangat tinggi)
i. Sangat tinggi di kompetensi, rendah di dimensi lain.
ii. Sangat tinggi di integritas, rendah di dimensi lain.
iii. Sangat tinggi di transparansi, rendah di dimensi lain
iv. Sangat tinggi di empati, rndah di dimensi lain

In [27]:
# --- 4. Trust Radar (4A) - Ketimpangan Dimensi Kepercayaan ---
# Logika:
# - Mengukur 4 dimensi kepercayaan: Expertise, Transparency, Empathy, Commitment.
# - Menghitung rata-rata skor untuk setiap dimensi berdasarkan kelompok pertanyaan terkait.
# - Mengidentifikasi responden yang memiliki pandangan "berat sebelah" (satu dimensi sangat tinggi, sisanya rendah).
#
# Output kolom utama:
#   - 'expertise', 'transparency', 'empathy', 'commitment':
#          Rata-rata skor per dimensi (skala numerik).
#   - 'gap_4a' :
#          Nilai ketimpangan, dihitung dengan rumus:
#          (Skor Dimensi Tertinggi) - (Rata-rata skor 3 dimensi lainnya).
#   - 'dominant_dim':
#          Nama dimensi yang memiliki skor tertinggi.
#
# Outlier criteria:
#   - Responden dianggap outlier bila:
#          -> gap_4a >= 2
#          (Artinya ada perbedaan signifikan antara aspek yang paling dipercaya vs aspek lainnya).
#
# Filter tambahan:
#   - Hanya menampilkan responden yang menjawab:
#          -> "Ya, bersedia" pada kolom contact_col
#
# Output final:
#   - Tabel berisi responden dengan pola kepercayaan yang tidak merata.
#   - Disortir descending berdasarkan 'gap_4a' (ketimpangan terbesar di atas).

In [28]:
"""
4. Trust Radar (4A)
"""

expertise_cols = [
    '1. Informasi yang disajikan oleh KPK akurat dan berbasis fakta',
    '2. Bahasa yang digunakan dalam publikasi KPK jelas dan mudah dipahami oleh publik',
    '3. KPK (melalui publikasinya) terlihat cepat dan tanggap dalam memberikan klarifikasi atas isu yang berkembang di masyarakat',
    '4. Informasi yang diberikan oleh media publikasi KPK bersifat mendalam dan informatif'
]

transparency_cols = [
    '6. KPK menyampaikan informasi secara konsisten di berbagai kanal komunikasinya (media sosial, website, konferensi pers)',
    '7. KPK memberikan penjelasan yang memadai tentang dasar keputusan atau langkah yang diambilnya',
    '8. KPK (melalui publikasinya) terlihat jujur dan tidak terlihat berusaha menutupi-nutupi informasi tertentu dari publik',
    '9. Informasi dan data (seperti Laporan Tahunan, Statistik Penindakan, Hasil Kajian, Peraturan) mudah ditemukan dan diakses di saluran resmi KPK'
]

empathy_cols = [
    '11. Isu-isu yang diangkat dalam publikasi KPK relevan dengan kekhawatiran saya sebagai masyarakat',
    '12. Melalui interaksi di media sosial atau salurannya, KPK terlihat mendengarkan dan menghargai masukan dari masyarakat',
    '13. Bahasa dan nada (tone) komunikasi KPK terasa \'membumi\' dan tidak arogan',
    '14. KPK terlihat berusaha mengajak masyarakat bersama-sama memahami dampak korupsi dalam kehidupan nyata'
]

commitment_cols = [
    '16. Pemberitaan dan publikasi KPK terasa independen dan bebas dari intervensi politik atau kepentingan lain',
    '17. KPK terlihat tegas dan tidak ragu dalam mengambil tindakan terhadap pihak yang terlibat korupsi (melalui publikasi resminya)',
    '18. Publikasi KPK mencerminkan komitmen yang tulus untuk memberantas korupsi demi kepentingan publik, bukan agenda lain',
    '19. Tindakan dan publikasi KPK terlihat konsisten dan tidak tebang pilih dalam menangani kasus'
]

all_4a_cols = expertise_cols + transparency_cols + empathy_cols + commitment_cols

# ubah format
df[all_4a_cols] = df[all_4a_cols].apply(pd.to_numeric, errors='coerce')

# mean
df['expertise']    = df[expertise_cols].mean(axis=1)
df['transparency'] = df[transparency_cols].mean(axis=1)
df['empathy']      = df[empathy_cols].mean(axis=1)
df['commitment']   = df[commitment_cols].mean(axis=1)

dims = ['expertise', 'transparency', 'empathy', 'commitment']
df[dims] = df[dims].astype(float)

# calculate score gap
def calculate_gap_4a(row):
    scores = row[dims]
    if scores.isnull().any():
        return 0.0

    sorted_scores = scores.sort_values(ascending=False)
    max_score = sorted_scores.iloc[0]
    avg_others = sorted_scores.iloc[1:].mean()

    return max_score - avg_others

df['gap_4a'] = df.apply(calculate_gap_4a, axis=1)
df['dominant_dim'] = df[dims].idxmax(axis=1)

GAP_THRESHOLD = 2  

outliers_4a_contactable = (
    df[(df['gap_4a'] >= GAP_THRESHOLD) & (df[contact_col] == 'Ya, bersedia')]
    .sort_values(by='gap_4a', ascending=False)
)

outliers_4a_contactable.head(20)
outliers_4a_contactable.to_excel('outliers_4a.xlsx', index=False)
